In [2]:
import matplotlib.animation as an
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import OffsetImage
import numpy as np
import openpyxl as px
from PIL import Image
from scipy.interpolate import CubicSpline

from constants2024 import NUMBER_OF_PLAYERS, NUMBER_OF_RACES, PLAYERS_TO_COLOURS, ROAD_COLOUR

In [3]:
wb = px.open(r"C:\Projekty\Coding\Python\F1PredsPPT\F12024 Predictions Tracking.xlsx",
    read_only = True,
    data_only = True)

In [4]:
score_history = wb["PrelimScoreMeanDelta"]

In [5]:
player_histories = {score_history.cell(2 + i, 1).internal_value:
                    [score_history.cell(2 + i, 2 + j).internal_value
                     for j in range(NUMBER_OF_RACES+1)]
                    for i in range(NUMBER_OF_PLAYERS)}

In [6]:
for player in player_histories:
    player_histories[player][0] = 0

In [7]:
interpolants = {name: CubicSpline(list(range(NUMBER_OF_RACES+1)), player_histories[name], 0, "clamped") for name in player_histories}

In [8]:
rs = np.linspace(0, NUMBER_OF_RACES, 1000)

In [9]:
for name in interpolants:
    player_colour = tuple(i / 255 for i in PLAYERS_TO_COLOURS[name])
    plt.plot(rs, interpolants[name](rs), color = player_colour, label = f"{name}")

plt.legend()
plt.grid()

In [10]:
def score_to_x(s):
    return 500 + 84/85 * (s - 250)

In [12]:
%matplotlib tk

fig, ax = plt.subplots()
ax.set_xbound(-250, 250)
ax.set_facecolor(ROAD_COLOUR)
ax.get_yaxis().set_visible(False)
ax.set_xlabel("Points from Mean")

player_images = {name: np.array(Image.open(rf"Cars2024/{name}.png")) for name in PLAYERS_TO_COLOURS}
artists = {name: OffsetImage(img) for name, img in player_images.items()}

fig.set_figheight(4.75)

name_to_y = {list(artists.keys())[i]: 75 + 50 * i for i in range(len(artists))}

for name in artists:
    artist = artists[name]
    ax.add_artist(artist)
    artist.set_offset((50, name_to_y[name]))

MAX_FRAME = 24 * 200

def t_at_frame(frame, max_frame = MAX_FRAME):
    t = frame / (max_frame) * 24
    return t

lines = ax.vlines(0, -1, 1, color = "black", linestyles="dashed")

def animate(frame):
    ax.clear()

    artists = {name: OffsetImage(img) for name, img in player_images.items()}
    t = t_at_frame(frame)
    name_to_w = {name: interpolant(t) for name, interpolant in interpolants.items()}
    ax.set_title(f"Round: {t:1f}")
    ax.set_xlabel("Points from Mean")

    r = t - int(t)
    xs = [-1 * 500 * r, -1 * 500 * r + 500]
    lines = ax.vlines(xs, -2, 2, color = "black", zorder = -1)
    ax.set_xbound(-250, 250)
    ax.set_ybound(-1, 1)

    for name in artists:
        ax.add_artist(artists[name])
        artists[name].set_offset((score_to_x(name_to_w[name]), name_to_y[name]))

anim = FuncAnimation(fig, animate, MAX_FRAME, repeat = False)

video = anim.save(
    "theAnimationNoLoop.gif",
    writer="pillow",
    fps = 50
)